# SARIMA

SARIMA stands for seasonal arima.it is family of box jenkin method. ARIMA has 3 differnt component together which is basically AR ( Auto regressive), MA( Moving average ) and I ( Integrating factor). Seasonal arima works like arima model but it has separate component to learn seasonality. seasonal arima has six hyperparameter p,d,q and P,D,Q which basically stands for AR,MA and intergating factor. later 3 terms is useful for seasonal factor. We can add external driver as well by exog and endog parameter.

Import all the requires packages. we are using statsmodel 0.10.0 latest stable version.need to be cautious while using date time.

In [1]:
#import all the packages...
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools
import warnings
warnings.filterwarnings("ignore")
from dateutil import relativedelta
import datetime

load data from excel file. we can add up workbook sheet as well in which we can import multiple sheets but here as single sheet is required hence directly read_excel works fine.

In [5]:
df = pd.read_excel('wholesaler_quarterly.xlsx')

In [6]:
df.describe()

,PayerCode,Volumn,avg_T,precipitation,Selling_days
count,7.207000e+03,7207.000000,7207.000000,7207.000000,7207.000000
mean,3.001022e+07,7570.700051,60.975458,3.498601,24.685861
std,4.974866e+03,12573.282372,14.107479,1.928127,1.089576
min,3.000051e+07,-0.090000,34.975083,0.380425,19.000000
25%,3.000523e+07,1826.147000,46.720930,1.693318,24.000000
50%,3.001100e+07,3585.606000,66.109886,3.674524,24.000000
75%,3.001494e+07,7646.167500,76.745568,5.597903,26.000000
max,3.001854e+07,135616.949000,82.476560,7.411308,27.000000


In [7]:

# df['VLC_vl'] = df['VLC']/df['Volume']
# X=df[['Date','Key','VLC_BGTRATE','VLC_vl']]
# df = X.copy()
#df = df[['Key','Date','Volume','avg_T','precipitation','PMI','Inflation_Rate','Selling_days_ratio']]
df.rename(columns={'PayerCode':'Key'},inplace=True)

In [30]:
# df = df.loc[df['Brand Family'] == 'BUD']
# df.drop(columns=['Brand Family'],axis=1,inplace=True)

In [ ]:
# X=df[['Date','Key','Volume','Selling_days','avg_T','precipitation','Consumer_Price_Index_month',
#       'Real_GDP_lcu_quarter','Unemployment_Rate_quarter','BGT_Volume']]

#X=df[['Date','Key','NR_Vol','VIC_Vol','VLC_Vol','MACO_Vol']]

In [ ]:
# X = df[['Date','Key','Volume','avg_T','precipitation','Industrial_Production_Index_month','Unemployment_Rate_quarter',
#          'Terms_of_Trade_annual','Exchange_Rate_Index_Period_Average_month','Exchange_Rate_LCU_per_US_Period_Average_month',
#         'Real_Consumption_Per_Capita_US_quarter','Wholesale_Producer_Price_Index_LCU_basus_month','Nominal_Private_Consumption_lcu_quarter',
#         'Consumer_Price_Index_month','Nominal_GDP_Per_Capita_US_quarter']]

#Sarima with leave and volume drivers..
#X = df[['Date','Key','Volume','GTO_Vol','Disc_Vol']]


# X = pd.concat([X,df[['Industrial_Production_Index_month','Unemployment_Rate_quarter','Terms_of_Trade_annual',
#                     'Consumer_Price_Index_month','Nominal_GDP_Per_Capita_US_quarter']]],axis=1)

#X = pd.concat([X,df[['Nominal_GDP_Per_Capita_US_quarter']]],axis=1)

In [ ]:
#holiday variables....
# X = pd.concat([X,df.iloc[:,697:]],axis=1)
# X = pd.concat([X,df.iloc[:,84:]],axis=1)

In [ ]:
#df = X.copy()

In [ ]:
#pd.pivot_table(df,index='Key',values=['NR_Vol','VIC_Vol','VLC_Vol','MACO_Vol'],aggfunc=['min','mean',np.median,'max']).T
#pd.pivot(df,columns='Key')

In [ ]:
# df['qtr_year'] = df.Date.dt.quarter.astype("str")+str("_")+df.Date.dt.year.astype("str")
# tmp = df.groupby(by=['Key','qtr_year'])['BGT_Volume'].sum()
# tmp = pd.DataFrame(tmp)
# tmp.columns = ['BGT_qtr_Vol']
# tmp.reset_index(inplace=True)
# df = pd.merge(df,tmp,how='left',on=['Key','qtr_year'])
# df.drop(columns=['qtr_year'],inplace=True)

In [ ]:
# df['year'] = df.Date.dt.year
# tmp = df.groupby(by=['Key','year'])['BGT_Volume'].sum()
# tmp = pd.DataFrame(tmp)
# tmp.columns = ['BGT_year_Vol']
# tmp.reset_index(inplace=True)
# df = pd.merge(df,tmp,how='left',on=['Key','year'])
# df.drop(columns=['year'],inplace=True)

In [ ]:
# df['qtr'] = df.Date.dt.quarter
# df['month'] = df.Date.dt.month

In [ ]:
# def bgt_transformation(brand_df):
#     brand_df['qtr_year'] = brand_df.Date.dt.quarter.astype(str)+str("_")+brand_df.Date.dt.year.astype(str)
#     brand_df['cum_sum_bgt_qtr'] = brand_df.groupby(by=['qtr_year'])['BGT_Volume'].cumsum()
#     brand_df['BGT_qtr_ratio']= brand_df['cum_sum_bgt_qtr'].divide(brand_df['BGT_qtr_Vol'])
#     brand_df['cum_sum_vol_qtr'] = brand_df.groupby(by=['qtr_year'])['Volume'].cumsum()
#     brand_df['cum_sum_vol_qtr_shift'] = brand_df['cum_sum_vol_qtr'].shift(1)
#     brand_df.loc[((brand_df.month==1)|(brand_df.month==4)|(brand_df.month==7)|(brand_df.month==10)),'cum_sum_vol_qtr_shift'] = 0
#     brand_df['bgt_diff_till_month'] = brand_df['BGT_qtr_Vol'] - brand_df['cum_sum_vol_qtr_shift']
#     brand_df.drop(columns=['qtr_year','cum_sum_bgt_qtr','cum_sum_vol_qtr','cum_sum_vol_qtr_shift'],inplace=True)
#     return brand_df

Convert date column in datetime format as we have to create index as datetime format only. be careful while giving format of date as it might change later operations. get the unique brand list as we have to iterate over all the brands separately

In [12]:
df.Date = pd.to_datetime(df.Date,format='%d-%m-%Y')
#Brand_list = ['BUD','DFH','GSB','HBBP','HBI','HBO','HKOW','ISP','OTH','SED']
#Brand_list = df.Key.unique()
Brand_list = [30000514]

In [13]:
Brand_list

[30000514]

In [ ]:
# for i,brand_key in enumerate(Brand_list):
#     Brand_list[i] = Brand_list[i].replace(' ','_')

In [8]:
# Brand_list = ['BUD_ALU_Anhui']

In [14]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

For modelling we follow differnt steps:
1. iterate over all brands and take out single brand each time.
2. provide train and test date based on requirement.
3. divide train and test data as le cycle works for whole year. ex apr is start so (3+9,4+8 etc..) each time upto december we have to complete, start date could be any.
4. iterate over p,d,q and P,D,Q value. provide min and max value which basically decide by ACF and PACF lag.
5. compare the aic and bic value and select least one. As least aic is best.
6. foorecast for the aic value
7. forecast for required no of steps ( basically ahead month or year based on freuqency).
8. save the result and iterate till end

In [8]:
final = pd.DataFrame(columns=["Brand","Forecast_values","Actual_values"])
for brand_name in Brand_list:
    print(brand_name)
    brand_df = df.loc[df.Key == brand_name]
    #brand_df = bgt_transformation(brand_df)
    brand_df.set_index('Date',inplace=True)
    tmp = []
    forecast = pd.DataFrame()
    Actuals = pd.DataFrame()
    #brand_df = brand_df[:'2019-12-01']
    s = pd.DataFrame()
    k = pd.DataFrame()
    
    if len(brand_df)>12:
        train_start = datetime.date(2019,1, 1)
        train_till = datetime.date(2019, 7, 1)
        Actuals_end = datetime.date(2020, 1, 1)
        train_date = train_start
        while train_date < train_till:
            test_date = train_date + relativedelta.relativedelta(months=1)
            dependent_colume = 'Volume'
            x = brand_df.drop(columns=[dependent_colume,'Key'])
            y = brand_df[[dependent_colume]]
            train_x = x[:train_date]
            train_y = y[:train_date][[dependent_colume]]
            test_x = x[test_date:]
            test_y = y[test_date:][[dependent_colume]]
            train_date = train_date + relativedelta.relativedelta(months=3)

            if ((test_date.month == 8) or (test_date.month == 2)):
                print("July imputation...")
                train_y[train_date:train_date] = np.nan

            p = d= q = range(0,2)
            # generate p,d,q value between 0 and 2
            pdq = list(itertools.product(p, d, q))
            seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
            pdq_list = {}
            spdq_list = {}
            aic_value = {}
            flag = 0
            for param in pdq:
                for param_seasonal in seasonal_pdq:
                    try:
                        mod = SARIMAX(exog=train_x,
                                      endog=train_y,
                                      order=param,
                                      seasonal_order=param_seasonal,
                                      enforce_stationarity=True,
                                      enforce_invertibility=True)

                        results = mod.fit()
                        if flag == 0:
                            aic_value['aic'] = results.aic
                            pdq_list['pdq'] = param
                            spdq_list['spdq_list'] = param_seasonal
                        else :
                            #check for least aic value and store pdq and seasonal parameters...
                            if aic_value['aic'] > results.aic :
                                aic_value['aic'] = results.aic
                                pdq_list['pdq'] = param
                                spdq_list['spdq_list'] = param_seasonal
                        flag = flag + 1
                    except:
                        continue

            #run sarima by best combination of parameters...
            try:
                mod = sm.tsa.statespace.SARIMAX(exog=train_x,
                                                endog=train_y,
                                                order=pdq_list['pdq'],
                                                seasonal_order=spdq_list['spdq_list'],
                                                enforce_stationarity=True,
                                                enforce_invertibility=True)
                results = mod.fit()
        #         #forecast for next month....
                if test_date > Actuals_end:
                    s = pd.DataFrame()
                    s["Forecast_values"] = list(results.get_prediction(exog=test_x[test_date:],start=test_x.index.min(),end=test_x.index.max()).predicted_mean.values)
                    s.index = test_y[test_date:].index
                    s["Brand"] = str(brand_name) +str("_")+ s.index.month.astype(str) +str("_")+s.index.year.astype(str)


                    k = pd.DataFrame()
                    k = test_y[test_date:]
                    k.columns = ['Actual_values']
                    k.index = test_y[test_date:].index
                    k["Brand"] = str(brand_name) +str("_")+ k.index.month.astype(str) +str("_")+k.index.year.astype(str)
                    break
                
                print(test_x.index.min())
                print(test_x[test_date:test_date])
                forecast[str(brand_name)+str('_')+str(test_date.month)+str("_")+str(test_date.year)] = results.get_prediction(exog=test_x[test_date:test_date],
                                                                                            start=test_x.index.min()).predicted_mean.values
                Actuals[str(brand_name)+str('_')+str(test_date.month)+str("_")+str(test_date.year)] = test_y[test_date:test_date].values[0]
           
            except Exception as e:
                print("in exception")
                continue
        
        if ((len(forecast)>0) & (len(Actuals)>0)):
            forecast=forecast.T.reset_index()
            forecast.columns=["Brand","Forecast_values"]
            if(len(s)>0):
                forecast= pd.concat([forecast,s],axis=0)
            Actuals=Actuals.T.reset_index()
            Actuals.columns=["Brand","Actual_values"]
            if(len(k)>0):
                Actuals= pd.concat([Actuals,k],axis=0)
            brand_wise_merge = forecast.merge(Actuals,on="Brand",how="left")
            final = final.append(brand_wise_merge,ignore_index=True)
        else:
            print("No forecast")
    else:
        print("length mismatch")

BUD
2019-04-01 00:00:00
              avg_T  precipitation      PMI  Inflation_Rate  \
Date                                                          
2019-04-01 62.51758        3.67452 50.10000         2.50000   

            Selling_days_ratio  
Date                            
2019-04-01             0.83333  
2019-05-01 00:00:00
              avg_T  precipitation      PMI  Inflation_Rate  \
Date                                                          
2019-05-01 69.70219        4.22272 49.40000         2.70000   

            Selling_days_ratio  
Date                            
2019-05-01             0.77419  


In [9]:
final.head(20)

,Brand,Forecast_values,Actual_values
0,BUD_4_2019,1825638.94175,1526338.82912
1,BUD_5_2019,2097858.27822,2070232.41412


Calculate the accuracy which is mape (1- ABS(actual-forcast)/actual). we use mape as it is unit less and later can multiply with 100 so we can get results in percentage

In [17]:
final['Error']=np.abs(final['Actual_values']-final['Forecast_values'])
final[['Brandname','leMonth','leYear']] = final.Brand.str.split("_",expand=True)

In [18]:
Agg_accuracy = final.groupby(by=['leMonth','leYear']).sum()[['Error','Actual_values','Forecast_values']]
Agg_accuracy['Accuracy'] = np.round((1-(Agg_accuracy['Error']/Agg_accuracy['Actual_values']))*100,2)

In [10]:
Agg_accuracy_brand = final.groupby(by=['Brandname']).sum()[['Error','Actual_values','Forecast_values']]
Agg_accuracy_brand['Accuracy'] = np.round((1-(Agg_accuracy_brand['Error']/Agg_accuracy_brand['Actual_values']))*100,2)

In [19]:
Agg_accuracy

Error  Actual_values  Forecast_values  Accuracy
leMonth leYear                                                        
1       2020    8.373235e+04   2.165093e+06     2.081361e+06     96.13
10      2019    1.830805e+05   1.228504e+06     1.411585e+06     85.10
        2020    1.005060e+06   0.000000e+00     1.005060e+06      -inf
11      2019    1.574344e+05   1.187562e+06     1.344997e+06     86.74
        2020    9.021242e+05   0.000000e+00     9.021242e+05      -inf
12      2019    2.583010e+03   1.196691e+06     1.194108e+06     99.78
        2020    8.498954e+05   0.000000e+00     8.498954e+05      -inf
2       2020    1.096673e+06   1.749663e+05     1.271639e+06   -526.79
3       2020    8.848868e+05   0.000000e+00     8.848868e+05      -inf
4       2019    2.993001e+05   1.526339e+06     1.825639e+06     80.39
        2020    1.312857e+06   0.000000e+00     1.312857e+06      -inf
5       2019    2.762586e+04   2.070232e+06     2.097858e+06     98.67
        2020    1.910922e+06   0.000000e+00     1.910922e+06      -inf
6       2019    2.848402e+05   2.367853e+06     2.083013e+06     87.97
        2020    2.132445e+06   0.000000e+00     2.132445e+06      -inf
7       2019    5.510126e+05   1.748441e+06     2.299453e+06     68.49
        2020    1.488841e+06   0.000000e+00     1.488841e+06      -inf
8       2019    3.054122e+05   1.965172e+06     1.659760e+06     84.46
        2020    1.738964e+06   0.000000e+00     1.738964e+06      -inf
9       2019    1.998547e+05   1.677322e+06     1.877176e+06     88.08
        2020    1.330234e+06   0.000000e+00     1.330234e+06      -inf

In [ ]:
Agg_accuracy

In [ ]:
# final[['Brand','Month']] = final.Brand.str.split("_",expand=True) 
# print(final)
# final['Error']=np.abs(final['Forecast_values']-final['Actual_values'])
# final_2=final.groupby(by=['Month','Brand'])['Error',"Actual_values"].sum().reset_index()
# final_2['Error_perc']=final_2['Error']/final_2['Actual_values']
# print(final_2)

In [46]:
final.to_excel('./SARIMA/sarima_Brand_pmi_inflation_seldayratio_temp_prec_Vol.xlsx')

In [ ]:
# final = pd.DataFrame(columns=["Brand","Forecast_values","Actual_values"])
# for brand_name in Brand_list:
#     print(brand_name)
#     brand_df = df.loc[df.Key == brand_name]
#    # brand_df = bgt_transformation(brand_df)
#     brand_df.set_index('Date',inplace=True)
#     tmp = []
#     forecast = pd.DataFrame()
#     Actuals = pd.DataFrame()
#     brand_df = brand_df[:'2019-12-01']
    
#     if len(brand_df)>12:
#         for i in range(9,0,-1):
#             train_date = brand_df.index.max() - relativedelta.relativedelta(months=i)
#             test_date = train_date + relativedelta.relativedelta(months=1)
            
#             dependent_colume = 'Volume'
#             x = brand_df.drop(columns=[dependent_colume,'Key'])
#             y = brand_df[[dependent_colume]]
#             train_x = x[:train_date]
#             train_y = y[:train_date][[dependent_colume]]
#             test_x = x[test_date:test_date]
#             test_y = y[test_date:test_date][[dependent_colume]]

#             if (test_date.month == 8):
#                 print("July imputation...")
#                 train_y[train_date:train_date] = np.nan

#             p = d= q = range(0,2)
#             # generate p,d,q value between 0 and 2
#             pdq = list(itertools.product(p, d, q))
#             seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
#             pdq_list = {}
#             spdq_list = {}
#             aic_value = {}
#             flag = 0
#             for param in pdq:
#                 for param_seasonal in seasonal_pdq:
#                     try:
#                         mod = SARIMAX(exog=train_x,
#                                       endog=train_y,
#                                       order=param,
#                                       seasonal_order=param_seasonal,
#                                       enforce_stationarity=True,
#                                       enforce_invertibility=True)

#                         results = mod.fit()
#                         if flag == 0:
#                             aic_value['aic'] = results.aic
#                             pdq_list['pdq'] = param
#                             spdq_list['spdq_list'] = param_seasonal
#                         else :
#                             #check for least aic value and store pdq and seasonal parameters...
#                             if aic_value['aic'] > results.aic :
#                                 aic_value['aic'] = results.aic
#                                 pdq_list['pdq'] = param
#                                 spdq_list['spdq_list'] = param_seasonal
#                         flag = flag + 1
#                     except:
#                         continue

#             #run sarima by best combination of parameters...
#             try:
#                 mod = sm.tsa.statespace.SARIMAX(exog=train_x,
#                                                 endog=train_y,
#                                                 order=pdq_list['pdq'],
#                                                 seasonal_order=spdq_list['spdq_list'],
#                                                 enforce_stationarity=True,
#                                                 enforce_invertibility=True)
#                 results = mod.fit()
#         #         #forecast for next month....
#                 print(results.get_prediction(exog=test_x[:test_date],start=test_x.index.min()).predicted_mean.values)
#                 forecast[str(brand_name)+str("_")+str(test_date.month)] = results.get_prediction(exog=test_x[:test_date],
#                                                                                             start=test_x.index.min()).predicted_mean.values
#                 Actuals[str(brand_name)+str("_")+str(test_date.month)] = test_y[:test_date].values[0]
#             except:
#                 continue
#         if (len(forecast)>0 & len(Actuals>0)):
#             forecast=forecast.T.reset_index()
#             forecast.columns=["Brand","Forecast_values"]
#             Actuals=Actuals.T.reset_index()
#             Actuals.columns=["Brand","Actual_values"]
#             brand_wise_merge = forecast.merge(Actuals,on="Brand",how="left")
#             final = final.append(brand_wise_merge,ignore_index=True)
#         else:
#             print("No forecast")
#     else:
#         print("length mismatch")